In [1]:
# Imports
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from datetime import date

In [2]:
# Definindo características da página
url = 'https://www.fundamentus.com.br/resultado.php'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

In [3]:
# Fazendo a request
try:
    req = Request(url, headers = headers)
    response = urlopen(req)
    print('Foi\n\n')
    print(response.getcode())
    html = response.read()
    # Tratando alguns possíveis erros
except HTTPError as e:
    print('HTTPError\n\n')
    print(response.getcode())
    print(e.status, e.reason)

except URLError as e:
    print('URLError\n\n')
    print(response.getcode())
    print(e.reason)

Foi


200


In [4]:
# Instanciando um objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Pegando os nomes das colunas da tabela
colunas_names = [col.getText() for col in soup.find('table', {'id': 'resultado'}).find('thead').findAll('th')]
colunas = {i: col.getText() for i, col in enumerate(soup.find('table', {'id': 'resultado'}).find('thead').findAll('th'))}

# Criando um DataFrame com os nomes das colunas
dados = pd.DataFrame(columns=colunas_names)

# Pegando os dados da tabela por linha
for i in range(len(soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr'))):
    linha = soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr')[i].getText().split('\n')[1:]
    inserir_linha = pd.DataFrame(linha).T.rename(columns=colunas)
    dados = pd.concat([dados, inserir_linha], ignore_index=True)

In [5]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import sqlalchemy
#import pyodbc 

In [6]:
SERVER = 'DESKTOP-KQ4D659'
DATABASE = 'PIPESOURCE_B3'
DRIVER = 'SQL Server Native Client 11.0'
USERNAME = ''
PASSWORD = ''
DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

In [7]:
engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

In [8]:
dados = dados.drop(columns=[21])
dados.rename(columns={'Cotação':'Cotacao',
'P/L':'PL',
'P/VP':'PVP',
'Div.Yield':'DivYield',
'P/Ativo':'P_Ativo',
'P/Cap.Giro':'P_CapitalGiro',
'P/EBIT':'P_Ebit',
'P/Ativ Circ.Liq':'P_Ativ_Circ_Liq',
'EV/EBIT':'Ev_Ebit',
'EV/EBITDA':'Ev_Ebitda',
'Mrg. Líq.':'MrgLiq',
'Liq. Corr.':'LiqCorr',
'Liq.2meses':'Liq2meses',
'Patrim. Líq':'PatrimLiq',
'Dív.Brut/ Patrim.':'DivBrutaPatrimonio',
'Cresc. Rec.5a':'CrescReceita5anos'}
, inplace = True)

In [9]:
from datetime import datetime as dt
# import re

dados.to_sql('PIPESOURCE_B3_FATO',con=engine,if_exists='replace')

time = dt.now()
time = str(time).replace(':','')[:15].replace(' ','_')
dados['inbound_date'] = time

# print(time)
dados.to_csv(f'C:/Projeto_B3/0 - Historico/fundamentus.csv',sep=';')